import necessary libs


In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [20]:
df=pd.read_csv('./dataset.csv')
df=df.drop('customerID',axis='columns')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df.isnull().any()

In [22]:
df['TotalCharges'] = df['TotalCharges'].replace(np.nan, df['TotalCharges'].mean())

In [29]:
numeric_columns = df.select_dtypes(exclude=['int64', 'float64']).columns
li=list(numeric_columns)
print("Numeric columns:",li )


Numeric columns: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']


In [30]:
lb=LabelEncoder()

for i in li:
    df[i]=lb.fit_transform(df[i])

df.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif

def analyze_feature_importance(df, target_col):
    # Separate features and target
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # 1. Correlation with target
    correlations = abs(df.corr()[target_col]).drop(target_col)
    corr_scores = correlations.sort_values(ascending=False)
    
    # 2. Random Forest Importance
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)
    rf_importance = pd.Series(rf.feature_importances_, index=X.columns)
    rf_scores = rf_importance.sort_values(ascending=False)
    
    # 3. Mutual Information
    mi_scores = mutual_info_classif(X, y)
    mi_series = pd.Series(mi_scores, index=X.columns)
    mi_scores = mi_series.sort_values(ascending=False)
    
    # 4. Combine all scores
    # Normalize each score type
    normalized_scores = pd.DataFrame({
        'Correlation': (corr_scores - corr_scores.min()) / (corr_scores.max() - corr_scores.min()),
        'RandomForest': (rf_scores - rf_scores.min()) / (rf_scores.max() - rf_scores.min()),
        'MutualInfo': (mi_scores - mi_scores.min()) / (mi_scores.max() - mi_scores.min())
    })
    
    # Calculate average importance
    normalized_scores['Average_Importance'] = normalized_scores.mean(axis=1)
    final_ranking = normalized_scores.sort_values('Average_Importance', ascending=False)
    
    return final_ranking


importance_ranking = analyze_feature_importance(df, 'Churn')
print("Top 5 most important features:")
print(importance_ranking.head(5))

Top 10 most important features:
                Correlation  RandomForest  MutualInfo  Average_Importance
tenure             0.870704      0.765767    0.799130            0.811867
Contract           1.000000      0.355409    1.000000            0.785136
TotalCharges       0.491665      1.000000    0.432643            0.641436
MonthlyCharges     0.476022      0.936737    0.469396            0.627385
OnlineSecurity     0.723258      0.190206    0.591985            0.501817


### Highly Relavent features
| Feature | Correlation | RandomForest | MutualInfo | Average_Importance |
|---|---|---|---|---|
| tenure | 0.870704 | 0.836014 | 0.770066 | 0.825594 |
| Contract | 1.000000 | 0.410168 | 1.000000 | 0.803389 |
| MonthlyCharges | 0.476022 | 0.967024 | 0.499967 | 0.647671 |
| TotalCharges | 0.491665 | 1.000000 | 0.444237 | 0.645301 |
| OnlineSecurity | 0.723258 | 0.219954 | 0.742850 | 0.562021 |
| TechSupport | 0.705692 | 0.209430 | 0.668060 | 0.527727 |

In [ ]:
df=df.drop(['OnlineBackup','DeviceProtection','PaymentMethod','PaperlessBilling','SeniorCitizen','Partner','gender'],axis=1)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,0,0,0,0,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,0,0,1,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,0,0,0,0,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,2,0,0,1,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,70.70,151.65,1


In [ ]:
# Get the top 5 feature names from your importance ranking
top_5_features = importance_ranking.head(5).index.tolist()

# Create final dataset with only top 5 features + target column
final_columns = top_5_features + ['Churn']
final_df = df[final_columns]

print("Selected features:", top_5_features)
print("\nFinal dataset shape:", final_df.shape)

Selected features: ['tenure', 'Contract', 'TotalCharges', 'MonthlyCharges', 'OnlineSecurity']

Final dataset shape: (7043, 6)


In [37]:
final_df.to_csv('final_dataset.csv', index=False)